# Multithreading scraper with using selenium and scrapy in Chrome

## Libraries downloading

In [ ]:
# download libraries
# scraping libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from fake_useragent import UserAgent

# multithreading
from concurrent.futures import ThreadPoolExecutor

# other
from datetime import date
import pandas as pd
import scrapy
import time
import numpy as np

from datetime import date
import datetime
from IPython.display import display
import re
import itertools

#for database connection
from dataclasses import dataclass
from sqlalchemy import create_engine, event, DateTime, Column, String, MetaData, Integer, \
    Binary, PrimaryKeyConstraint, Date
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, Session
from contextlib import contextmanager
from urllib.parse import quote_plus

## Functions definition

In [ ]:
def get_driver(chromedriver_path: str) -> webdriver.Chrome:
    """Returns Chromedriver with specified parameters"""
    options = webdriver.ChromeOptions()
    
    # headless-mode activation
    options.headless = True
    options.add_argument('window-size=1920x1080')
    ###############################################
    
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)

    #fix handshake failed
    options.add_argument("--ignore-certificate-error")
    options.add_argument("--ignore-ssl-errors")
    caps = webdriver.DesiredCapabilities.CHROME.copy()
    caps['acceptInsecureCerts'] = True
    caps['acceptSslCerts'] = True
    
    #add user-agent to avoid blocking
    #create random user-agents
    ua = UserAgent()
    userAgent = ua.random
    
    #add user-agent to options
    options.add_argument(f'user-agent={userAgent}')

    
    #set driver parametres
    driver = webdriver.Chrome(options=options,
                              executable_path=chromedriver_path,
                              desired_capabilities=caps)
    
    # uncomment in non-headless mode
    #driver.maximize_window()
    
    driver.implicitly_wait(15)
    return driver

    
def choose_city(city_name: str, driver: webdriver.Chrome) -> None:
    """Chooses the necessary city"""
    if city_name == "Москва":
        driver.find_element_by_xpath('//*[contains(text(), "Да, все верно")]').click()
        time.sleep(np.random.choice(delays))
    else:
        driver.find_element_by_xpath('//*[contains(text(), "Выбрать другой")]').click()
        time.sleep(np.random.choice(delays))

        # push choose city button
        driver.find_element_by_xpath(f'//*[contains(text(), "{city_name}")]').click()
        time.sleep(np.random.choice(delays))

        
def choose_store(store: str, driver: webdriver.Chrome) -> None:
    """Chooses the necessary store address"""
    try:
        #open the choose store window
        driver.find_element_by_xpath('//*[contains(text(), "Выбрать другой")]').click()
        time.sleep(np.random.choice(delays))
 
        #find address
        driver.find_element_by_xpath(f'//div[@class="store-item__content"]/div[contains(text(), "{store}")]').click()
        time.sleep(np.random.choice(delays))
        
        #confirm selection
        driver.find_element_by_xpath(f'//span[contains(text(), "Выбрать магазин")]').click()
        time.sleep(np.random.choice(delays))
        
    except Exception as e:
        print('Error in choosing the store')
        print(e)
    
    
def close_cookie_window(driver: webdriver.Chrome) -> None:
    '''Closes the cookie window'''
    try:
        driver.find_element_by_xpath('//div[@class="button__inner cookie-usage-notice__button-inner--desktop"]').click()
        time.sleep(np.random.choice(delays))
    except:
        pass
    
    
def load_full_page(driver: webdriver.Chrome) -> None:
    """Clicks 'Загрузить ещё' button as many times as needed to load full page"""
    while True:
        try:
            button_element = driver.find_element_by_xpath("//*[contains(text(), 'Загрузить ещё')]")
            button_element.click()
            time.sleep(1)
        except:
            try:
                # print('1st Error in full page loading')
                button_element = driver.find_element_by_xpath("//*[contains(text(), 'Загрузить ещё')]")
                button_element.click()
                time.sleep(1)
            except:
                pass
            # print('Page downloaded')
            break

def get_categories_and_links(driver: webdriver.Chrome) -> None:
    """Gets category name and link from categories page"""
    try:
        find_element_by_xpath('/html/body/div[1]/div[1]/div/main/article/div/div/div/div[2]/a')
        all_types_element = driver.find_elements_by_xpath('/html/body/div[1]/div[1]/div/main/article/div/div/div/div[2]/a')
    except:
        #if there is a discount window on the page
        all_types_element = driver.find_elements_by_xpath('/html/body/div[1]/div[1]/div/main/article/div/div/div/div[3]/a')
    
    all_types = [(prod_type.text, prod_type.get_attribute('href')) for prod_type in all_types_element]
    
    return all_types

def get_cities_categories(inp: tuple) -> None:
    """Gets category name and link from categories page for several cities"""
    city, store = inp #city and store address as input
    
    # open browser
    driver = get_driver(chromedriver_path='/Program Files (x86)/chromedriver')
    
    # go to catalog page
    driver.get(categories_link)
    
    time.sleep(np.random.choice(delays))
    
    # Choose the necessary location
    # Choose city
    choose_city(city, driver)

    # Choose store
    choose_store(store, driver)

    # Close cookie window
    close_cookie_window(driver)

    # Get categories
    all_types = get_categories_and_links(driver)
    
    # Add city and store to categories and link
    all_product_links = [(prod_name, prod_link, city, store) for prod_name, prod_link in all_types]
    
    stores_categories.append(all_product_links)

    print(city, store,'finished with',len(all_product_links),'categories')
    
    # close browser    
    driver.quit()

def collect_data(html_str: str, xpaths: dict, prod_type: str, city: str, store: str) -> pd.DataFrame:
    """Collect the necessary data for each product on page"""
    selector = scrapy.Selector(text=html_str)
    products = selector.xpath("//div[@class='catalog-grid-container__grid']/div[@class='sku-card-small-container']")
    
    products_data = []

    for product in products:
        d = {}
        for name, xpath in xpaths.items():
            try:
                d[name] = product.xpath(xpath).get()
            except:
                pass

        if 'links' in d and d['links']:
            #d['links'] = 'link' + d['links']

        if 'item' in d and d['item']:
            d['item'] = d['item'].split('-')[-1][: -1]

        products_data.append(d)
    
    df = pd.DataFrame(products_data)
    
    #add category
    df['category'] = prod_type
    
    #add city
    df['city'] = city

    #add store
    df['store'] = store
    
    return df


def get_data(inp: tuple) -> None: #  city: str, store: str
    '''Gets products data for each category'''
    prod_type, link, city, store = inp

    # open browser
    driver = get_driver(chromedriver_path='/Program Files (x86)/chromedriver')

    # go to catalog page
    driver.get(link)
    time.sleep(np.random.choice(delays))
    
    # Choose location
    # Choose city
    choose_city(city, driver)
    
    # Choose store
    choose_store(store, driver)

    # Close cookie
    close_cookie_window(driver)
    
    # Load full page
    load_full_page(driver)
            
    # Get necessary data by xpaths
    xpaths = {'name': './/div[@class="sku-card-small__title"]//text()',
             'links': './a/@href',
             'item': './a/@href',
             'prices_int_reg': './/div[contains(@class, "sku-price--regular")]/span[@class="sku-price__integer"]/text()',
             'prices_fract_reg': './/div[contains(@class, "sku-price--regular")]/small[@class="sku-price__fraction"]/text()',
             'prices_int_disc': './/div[contains(@class, "sku-price--primary")]/span[@class="sku-price__integer"]/text()',
             'prices_fract_disc': './/div[contains(@class, "sku-price--primary")]/small[@class="sku-price__fraction"]/text()',
             'aktion_mark': './/div[contains(@class, "sku-card-small__discount-label")]/text()'}

    df = collect_data(driver.page_source, xpaths, prod_type, city, store)
    
    # close browser
    driver.quit()
    
    
    whole_data.append(df)
    
    print(f'Finished {prod_type} {city} {store} with {df.shape[0]} products')

## Setting parametres

In [ ]:
#list of stores addresses
#stores_list = [('city','store')]

#catalog page link
#categories_link = 'link'

#list of random delays
delays = [3,4,5,6,7,8,9,10]

## Getting categories for each store

In [ ]:
#get categories
stores_categories = []

start = datetime.datetime.now()

with ThreadPoolExecutor(max_workers=2) as executor:
    for store_data in stores_list:
        executor.submit(get_cities_categories, store_data)

#transformation
stores_categories = list(itertools.chain(*stores_categories))
        
#time stats   
finish = datetime.datetime.now()
print("Download complete. Passed: ", (finish - start).seconds//3600, "hours ", ((finish - start).seconds//60)%60, "minutes")
print()

print(stores_categories)

## Getting product data for each category in each store

In [ ]:
whole_data = []

start = datetime.datetime.now()

with ThreadPoolExecutor(max_workers=2) as executor:
    for product_data in stores_categories:
        executor.submit(get_data, product_data)
        
#stats    
finish = datetime.datetime.now()
print("Download complete. Passed: ", (finish - start).seconds//3600, "hours ", ((finish - start).seconds//60)%60, "minutes")
print('Categories: ',len(whole_data), 'of', len(stores_categories))

## Postprocessing

In [ ]:
#make dataframe
df = pd.concat(whole_data, axis=0).reset_index(drop=True)
#df = resp


#drop garbage from prices
df['prices_int_reg'] = df['prices_int_reg'].apply(lambda x: re.sub(re.compile(r'\s+'), '', x))
df['prices_int_disc'] = df['prices_int_disc'].apply(lambda x: re.sub(re.compile(r'\s+'), '', x))

#create regular price
#regular price
df['regular_price'] = df['prices_int_reg'].str.cat(df['prices_fract_reg'].values,sep='.')
df['regular_price'] = df['regular_price'].astype(float)

#discount price
df['promo_price'] = df['prices_int_disc'].str.cat(df['prices_fract_disc'].values,sep='.')
df['promo_price'] = df['promo_price'].astype(float)

#garbage out
df = df.drop(['prices_int_reg','prices_fract_reg','prices_int_disc','prices_fract_disc'], 1)

#make action mark
df['aktion'] = df['aktion_mark'].apply(lambda x: "Yes" if x != '' else '')
df = df.drop('aktion_mark', 1)

#add download date
df['download_date'] = date.today().strftime("%Y-%m-%d")

#reorder columns
df = df[['item', 'name', 'regular_price', 'promo_price','aktion', 'category', 'download_date', 'city', 'store', 'links']]

#create store id
df['store_id'] = df['city'].map(str) + '_' + df['store'].map(str)

#save data
for store_id in df['store_id'].unique():
    df_temp = df.query('store_id == @store_id')
    df_temp.drop('store_id', axis = 1).to_excel(store_id + '_' + datetime.datetime.now().strftime("%B%d") + '_full' +'.xlsx'
            , encoding ="UTF-8",index=False)
print('data saved')

## Database connection

Add parsing results into database

### Connection parametres

In [ ]:
#@dataclass(frozen=True)
#class DbConfig:
#    db_server = server address
#    pwd = password
#    uid = user id
#    db_name = database name
#    driver = driver
#    params = quote_plus(
#       'DRIVER={DRIVER};SERVER={DB_SERVER};DATABASE={DB_NAME};UID={UID};PWD={PWD}'.format(
#           DB_SERVER=db_server, DB_NAME=db_name, UID=uid, PWD=pwd, DRIVER=driver, 
#        ))
#    print(params)
#    conn_str = 'mssql+pyodbc:///?odbc_connect={}'.format(params)


#class Db:
#    def __init__(self):
#        self._db_conf = DbConfig()
#        self.engine = None

#    def create_engine(self):
#        if self.engine is None:
#            self.engine = create_engine(self._db_conf.conn_str)

#        @event.listens_for(self.engine, 'before_cursor_execute')
#        def receive_before_cursor_execute(conn, cursor, statement, params, context, executemany):
#            if executemany:
#                cursor.fast_executemany = True
#                cursor.commit()

#    @contextmanager
#    def open_session(self):
#        """Provide a transactional scope around a series of operations."""
#        session: Session = sessionmaker(bind=self.engine)()
#        try:
#            yield session
#            session.commit()
#        except:
#            session.rollback()
#            raise
#        finally:
#            session.close()

### Save results

In [ ]:
#df.drop('store_id', axis = 1).to_sql(name='' , con=db.engine, if_exists="append", schema='', index=False,
#                                    chunksize=10000)
#print('database updated')